<a href="https://colab.research.google.com/github/fedhere/UInotebooks/blob/master/imgPorcessingABC/aenc_faces_instuctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

author FBB for ETHICS OF DS 2021

This exercise is designed to think about bias in facial recognition software. 
To better understand how mathematical models can be biased, you will create a model that can increase the resolution of the image and try to identify in which cases the model underperforms and why.

# Autoencoder notebook 
create superresolution portraits from low resolution black and white images:
- load a patch of images and lower the resolution
- build an autoencoder with a bottleneck layer much smaller thn the number of pixels in an image
- pass the lower resolution images as imput and the higher resolution images as output
- ask the encoder to predict (generate) high resolution images from low resolution ones
- test it on a headshot of yourself

In [ ]:
# this will connect this notebook to your google drive. 
# You will need to approve the mounting point by choosing the account and 
# getting a code to be paster into this cell 
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
# this loads the keras library for deep learning
from keras.models import Sequential
from keras.layers import Dense#, Dropout, Flatten
from keras import backend as K
from keras.optimizers import Adamax


In [ ]:
# this loads other functions
import glob # to look for date in the folders on the drive
import h5py # to save DL models after training and read saved models
from PIL import Image # to interact with images
import numpy as np # for mathematical operations in python
import pylab as pl # to make plots
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
cd /content/gdrive/


you can choose a different destination than the one below, but if you do then you need to download the data. make sure you uncomment the ```wget``` line of code below and the ones that follow as appropriate

In [ ]:
cd Shareddrives/MLTSA2020/

/content/gdrive/Shareddrives/MLTSA2020


In [ ]:
!pwd

/content/gdrive/Shareddrives/MLTSA2020


In [ ]:
!ls

# load head shots from original data and save as numpy array - if you want you can skip to the "Read in data" section
faces95.zip is a compressed folder with headshots. It was removed from the internet after the "Obama Whitening Debackle". The code that follow was the original code to grab the data.  I have saved it and put it in a share drive that you should have access too. 

In [ ]:
##DO NOT RUN:
## this code no longer works cause the endpoint was removed
#!wget https://cswww.essex.ac.uk/mv/allfaces/faces95.zip

In [ ]:
##DO NOT RUN:
## the file is already unzipped in the shared drive so you do not need to run this
#!unzip faces95\ \(1\).zip

In [ ]:
## I saved the processed file but if you want you can uncomment the cells below to redo it
!ls faces95/*/*jpg

In [ ]:
## there are 1400 images so this is slow
flist = glob.glob("faces95/*/*jpg")
flist.remove([f for f in flist if "prima" in f ][0])


In [ ]:
N = len(flist)
N

1440

In [ ]:
## this saves the data into a numpy array
im = Image.open(flist[0])
np_im = numpy.array(im)[:,:,:3].astype(float) / 255 #normalize the images so that they are 0-1
pl.imshow(np_im)
## I am going to need only a subset of the image: the center part where the head is

np_im[25:165:2,25:165:2,:1].shape, np.prod(np_im.shape[1:]);

In [ ]:
## skip and read the saved version if you have saved it (after the first time you run this cell)
imgs_orig = np.zeros((N, 70, 70, 1))
for i,f in enumerate(flist):
   if "prima" in f: continue
   if not i%50: print(i)

   im = Image.open(f)
   np_im = numpy.array(im)[:,:,:3].astype(float) / 255 
   # here I collect the certain portion of each image 
   # from pixel 25 to 165 on the x and y axis (25:165)
   # and subsampling by a factor 2 (25:165:2)
   # and I am only choosing 1 color layer: [...,:1]: the R in RGB layers
   imgs_orig[i] = numpy.array(im)[25:165:2,25:165:2,:1].astype(float) / 255 
#np.save("faces2.npy", imgs)


## read in data

I saved the data in a numpy array: this way you can read it in more quickly. 

In [ ]:
#imgs_orig = np.load("faces2.npy")

In [ ]:
print("""There are {} images 
each {}x{} pixels and {} color channels""".format(*imgs_orig.shape))

# create input-output pairs
we want to create a model that can guess how a high resolution version of a low resolution headshot would look. We need to give it examples of low-res--high-res pairs. To do that we artificially lower the resolution of some images.

## make a copy of the original before you lower the resolution#

In [ ]:
#imgs orig is the full resolution image
imgs = imgs_orig.copy()

In [ ]:
# this creates a lower the resolution version of the image: reads every other pixel
imgs = imgs[:,::2,::2]
imgs[0].shape

In [ ]:
print("""There {} low resolution images are 
each {}x{} pixels and {} color channels""".format(*imgs.shape))

In [ ]:
print("The number of pixels in each image is {}".format(np.prod(imgs[0].shape)))

## generate a train and test set
The target variable will be a full resolution image set and the low input the lower resolution one

An autoencoder is traditionally trained to learn how to reproduce itself: input=target

We want to create a model that can increase the resolution of images: we will give the low resolution images as input and the high resolution image counterpart as target. Hovering over the function you can see what arguments it takes:

```train_test_split(input, target, test_size, random_state)```

In [ ]:
from sklearn.model_selection import train_test_split
X_tr,  X_te, y_tr, y_te = train_test_split(# pass the input and target: the low res and the high res set
                                             test_size = .05, random_state=1)


In [ ]:
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((1368, 35, 35, 1), (72, 35, 35, 1), (1368, 70, 70, 1), (72, 70, 70, 1))

In [ ]:
ax = pl.figure(figsize=(10,5)).subplots(1,2)
ax[0].imshow(np.squeeze(X_tr[0]), cmap="bone");
ax[1].imshow(np.squeeze(y_tr[0]), cmap="bone");

In [ ]:
ax = pl.figure(figsize=(10,5)).subplots(1,2)
ax[0].imshow(np.squeeze(X_tr[1]), cmap="bone");
ax[1].imshow(np.squeeze(y_tr[1]), cmap="bone");

In [ ]:
# this cell of code flattens the images: puts all pixels in a row
# this is apprioriate if the neural network does not use convolutional layers
X_tr = X_tr.reshape(len(X_tr), np.prod(X_tr.shape[1:]))
X_te = X_te.reshape(len(X_te), np.prod(X_te.shape[1:]))
y_tr = y_tr.reshape(len(y_tr), np.prod(y_tr.shape[1:]))
y_te = y_te.reshape(len(y_te), np.prod(y_te.shape[1:]))

In [ ]:
input_dim = X_tr[0].shape

# build an autoencoder model

model.add([....] 
## add Dense layers - type Dense to see how to use the function to create a layer
## remember that the network should have an hour glass shape


In [ ]:
# instantiate a sequential model: a deep neural network
model = Sequential()

#Encoder: reduces the size of the original image
model.add(# add Dense layers - type Dense to see how to use the function to create a layer
model.add(
    
# choose the optimizer and the learning rate
opt = Adamax(
    learning_rate=0.005)

#compile the model choosing the loss and the metrics to keep track of
model.compile(optimizer=opt, loss='mse',
              metrics=['mae', 'acc'])
#alternative options for optimizing optimizer="adadelta", loss="kullback_leibler_divergence")

In [ ]:
print(model.summary())

## train model OR you can skip and load the pretrained I saved for you!
or to read in the saved model skip this cell, or skip the whole section and read a model from http://fbb.space/mltsa/imgreconstruct_10000.h5

In [ ]:
# only use this if you are in your own folder
# !mkdir facecheckpoints

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback, EarlyStopping
# this callback allows us to not to wast training time:
# if the validation loss does not improve for N epochs in a row training stops
patience = EarlyStopping(patience=20)
# other callbacks that can be used to save intermediate training steps - only use if you are in your own folder
# filepath = "facecheckpoints/OneHotEncodedweights-improvement-{epoch:02d}-{loss:.4f}.hd5" 
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, 
#                              save_best_only=True, mode='min')


In [ ]:
# you can skip this and load the saved model
history = model.fit(
    #....
    #pass the input and output data as first and second arguments
    epochs=1000, #with the patience callback I can set lots of epochs 
                    validation_split=0.2,
                    batch_size=150, verbose=1,
                    callbacks=[patience])#, checkpoint])
#model.save("facecheckpoints/imgreconstruct2021_100000.h5")

In [ ]:
pl.plot(np.array(history.history['loss']), label="loss")
pl.plot(np.array(history.history['val_loss']), label="validatoin loss")
pl.xlabel("epochs")
pl.yscale('log')
pl.legend();
#pl.xscale('log')


In [ ]:
## READ IN THE SAVED MODEL
##!wget http://fbb.space/mltsa/imgreconstruct_10000.h5
#model.load_weights("facecheckpoints/imgreconstruct2021_100000.h5")

## predict

now that the model is trained it can be used to make prediction: this for us mean to crete high resolution image versions of low resolution imputs

In [ ]:
outim = model.predict(X_te)
# the output if a high res image for each image in input
outim.shape

(72, 4900)

In [ ]:
# to be read as images the predictions have to be organized in pizel rows and columns
outim[0].reshape(imgs_orig[0].shape).shape

(70, 70, 1)

In [ ]:
import scipy
def compareinout(i, outim, X_te):
  '''function to plot the input, the prediction and the target in 3 columns
  plots the ith training datum'''
  fig = pl.figure(figsize(10,5))
  ax = fig.add_subplot(131) 
  ax.imshow(X_te[i].reshape(imgs[i].shape[:2]) , cmap="bone")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  ax = fig.add_subplot(132) 
  ax.imshow(outim[i].reshape(imgs_orig[i].shape[:2]) , cmap="bone")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  ax = fig.add_subplot(133) 
  ax.imshow(y_te[i].reshape(imgs_orig[i].shape[:2]) , cmap="bone")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  print("similarity {:.2f}".format(scipy.stats.entropy(outim[i], y_te[i])))


plot some input-outpu pairs and the predictions

In [ ]:
np.random.seed(123)
# pick a few random test images
for i in range(30):
  j = np.random.randint(0, len(X_te))
  print(i + 1, ":\tinput\t\t   prediction\t\ttarget")
  compareinout(j, outim, X_te)
  pl.show()

What do you think? some are pretty good right?! e.g. 1, 4, 6, 7, 8

Some are really bad. 9... why?

inspet the imput data: for each of 72 persons in the dataset there are 20 images. inspect one per person

In [ ]:
#folders = glob.glob("faces95/*")
axs = pl.figure(figsize=(15,15)).subplots(9,8)

for i,f in enumerate(folders):
  #im = Image.open(f + "/1.jpg")
  np_im = numpy.array(imgs_orig[i*20].reshape(imgs_orig[0].shape[:-1])) 
  axs[i//8][i%8].imshow(np_im, cmap='bone')
  axs[i//8][i%8].axis('off')

# test the model on your own image
 depending on the initial size you will have to downsample the image, and extract a single color layer

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!ls

In [ ]:
imme = Image.open(YOUR IMAGE NAME)

In [ ]:
numpy.array(imme).shape

In [ ]:
pl.imshow(numpy.array(imme))

**figure out how to shape it to make it the right number of pixels**

In [ ]:
Nsamp = 

In [ ]:
testimage = numpy.array(imme)[:Nsamp*35,450:Nsamp*35+450, :3].mean(axis=-1)
pl.imshow(testimage, cmap="bone")

In [ ]:
print(35*Nsamp)

1050


low resolution version
subsampling by 25

In [ ]:
pl.imshow(testimage[::Nsamp,::Nsamp], cmap="bone")

In [ ]:
pl.imshow(model.predict(np.array([testimage[::Nsamp,::Nsamp].flatten() * 1.0 / 
                                  testimage[::Nsamp,::Nsamp].max()])).reshape(70,70), 
          cmap="bone")
pl.axis('off');